In [1]:
import re
import pandas as pd 
from urllib.parse import urlparse, parse_qs, urlunparse, urlencode
import random
import ast
import os 

In [7]:

links = pd.read_csv('/Users/alisa/Desktop/PittNAIL/cleardashboard_reviews/webscraping/amazon-devices.csv')
links.head()




,Unique ID,Product Name,Model,Company,search,amazon_link
0,1,Jabra Enhance,Enhance Select 200 Premium,GN Resound,Jabra EnhanceEnhance Select 200 Premium,NaN
1,2,Jabra Enhance,Enhance Select 200 Basic,GN Resound,Jabra EnhanceEnhance Select 200 Basic,NaN
2,3,Jabra Enhance,Enhance Select 100 Premium,GN Resound,Jabra EnhanceEnhance Select 100 Premium,NaN
3,4,Jabra Enhance,Enhance Select 100 Basic,GN Resound,Jabra EnhanceEnhance Select 100 Basic,NaN
4,5,Jabra Enhance,Enhance Select 50 Premium,GN Resound,Jabra EnhanceEnhance Select 50 Premium,NaN


In [8]:

available = links[links['amazon_link'].notna()]
available.head()
# for index, row in links:
#     if row.loc['amazon_link'] 


,Unique ID,Product Name,Model,Company,search,amazon_link
11,8,Eargo,Eargo 6,Eargo 6,EargoEargo 6,https://www.amazon.com/EARGO-Hearing-Aids-Adul...
12,9,Eargo,Eargo 7,Eargo 7,EargoEargo 7,https://www.amazon.com/EARGO-Self-Fitting-FDA-...
14,11,Lexie,Lexie B1,hearX Group,LexieLexie B1,https://www.amazon.com/Lexie-Hearing-Aids-Powe...
15,12,Lexie,Lexie B2,hearX Group,LexieLexie B2,https://www.amazon.com/Lexie-Rechargeable-Self...
16,13,IQbuds2 MAX,IQbuds2 MAX,Nuheara Limited,IQbuds2 MAXIQbuds2 MAX,https://www.amazon.com/Nuheara-Bluetooth-Recha...


### example of what links look like
 
1. a. main link (original)
https://www.amazon.com/EARGO-Hearing-Aids-Adults-Rechargeable/dp/B09NWCSS86/ref=sr_1_3?crid=2BYS6L6MUXD9E&dib=eyJ2IjoiMSJ9.97zAj3njr0szVavz8ZtBMro9lLRtBRtSUYiYQYBFP7rGjHj071QN20LucGBJIEps.cJnBb4WA30KdDKHDt26VhssSrvoUiI2KtarqecgjjxQ&dib_tag=se&keywords=eargo+ergo+5&qid=1719607779&sprefix=eargoeargo+5%2Caps%2C98&sr=8-3

1. b. main link (with long end trimmed)
https://www.amazon.com/EARGO-Hearing-Aids-Adults-Rechargeable/dp/B09NWCSS86/


2. first page of all reviews 
https://www.amazon.com/EARGO-Hearing-Aids-Adults-Rechargeable/product-reviews/B09NWCSS86/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews	

3. second page 
https://www.amazon.com/EARGO-Hearing-Aids-Adults-Rechargeable/product-reviews/B09NWCSS86/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=2

4. third page 
https://www.amazon.com/EARGO-Hearing-Aids-Adults-Rechargeable/product-reviews/B09NWCSS86/ref=cm_cr_getr_d_paging_btm_next_3?ie=UTF8&reviewerType=all_reviews&pageNumber=3 


In [7]:
def shorten_amazon_url(original_link):
    parsed_url = urlparse(original_link)

    path_parts = parsed_url.path.split('/') 
    # print(path_parts)
    if len(path_parts) >= 4:
        product_name = path_parts[1]
        product_id = path_parts[3]

        # print(product_name , " " ,product_id)
        main_product_link = f"{parsed_url.scheme}://{parsed_url.netloc}/{product_name}/dp/{product_id}/"
        # print(main_product_link)
        # https://www.amazon.com/EARGO-Hearing-Aids-Adults-Rechargeable/dp/B09NWCSS86/
        return main_product_link, product_name, product_id
    return "empty"
    


In [8]:
available = available.copy()

# shorten_amazon_url(available['amazon_link'].iloc[6])

In [9]:
available.loc[:, 'short_url'] = available['amazon_link'].apply(lambda x: shorten_amazon_url(x)[0])
available.loc[:, 'product_name'] = available['amazon_link'].apply(lambda x: shorten_amazon_url(x)[1])
available.loc[:, 'product_id'] = available['amazon_link'].apply(lambda x: shorten_amazon_url(x)[2])


available.head()
# available['Unique ID'].iloc[6]

,Unique ID,Product Name,Model,Company,search,amazon_link,short_url,product_name,product_id
11,8,Eargo,Eargo 6,Eargo 6,EargoEargo 6,https://www.amazon.com/EARGO-Hearing-Aids-Adul...,https://www.amazon.com/EARGO-Hearing-Aids-Adul...,EARGO-Hearing-Aids-Adults-Rechargeable,B09NWCSS86
12,9,Eargo,Eargo 7,Eargo 7,EargoEargo 7,https://www.amazon.com/EARGO-Self-Fitting-FDA-...,https://www.amazon.com/EARGO-Self-Fitting-FDA-...,EARGO-Self-Fitting-FDA-Cleared-Hearing-Aids,B0BSHCNP5W
14,11,Lexie,Lexie B1,hearX Group,LexieLexie B1,https://www.amazon.com/Lexie-Hearing-Aids-Powe...,https://www.amazon.com/Lexie-Hearing-Aids-Powe...,Lexie-Hearing-Aids-Powered-Bluetooth-enabled,B0BHHJWPH7
15,12,Lexie,Lexie B2,hearX Group,LexieLexie B2,https://www.amazon.com/Lexie-Rechargeable-Self...,https://www.amazon.com/Lexie-Rechargeable-Self...,Lexie-Rechargeable-Self-fitting-Invisible-Redu...,B0BJ5VBY94
16,13,IQbuds2 MAX,IQbuds2 MAX,Nuheara Limited,IQbuds2 MAXIQbuds2 MAX,https://www.amazon.com/Nuheara-Bluetooth-Recha...,https://www.amazon.com/Nuheara-Bluetooth-Recha...,Nuheara-Bluetooth-Rechargeable-Personalization...,B09J2PVRKC


In [10]:
available = available.copy()



2. first page of all reviews 
https://www.amazon.com/EARGO-Hearing-Aids-Adults-Rechargeable/product-reviews/B09NWCSS86/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews	

In [11]:
def generate_first_page_reviews_link(short_link):
    
    parsed_url = urlparse(short_link)

    path_parts = parsed_url.path.split('/') 
    # print(path_parts)
    product_name = path_parts[1]
    product_id = path_parts[3]
    
    review_link = f"{parsed_url.scheme}://{parsed_url.netloc}/{product_name}/product-reviews/{product_id}/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"
    return review_link

# generate_first_page_reviews_link('https://www.amazon.com/EARGO-Hearing-Aids-Adults-Rechargeable/dp/B09NWCSS86/')


In [12]:
available.loc[:, 'review1'] = available['short_url'].apply(generate_first_page_reviews_link)


first page: 
https://www.amazon.com/EARGO-Hearing-Aids-Adults-Rechargeable/product-reviews/B09NWCSS86/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews

3. second page 
https://www.amazon.com/EARGO-Hearing-Aids-Adults-Rechargeable/product-reviews/B09NWCSS86/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=2

4. third page 
https://www.amazon.com/EARGO-Hearing-Aids-Adults-Rechargeable/product-reviews/B09NWCSS86/ref=cm_cr_getr_d_paging_btm_next_3?ie=UTF8&reviewerType=all_reviews&pageNumber=3 


In [13]:
# list of review pages 2-10

def generate_reviews2_10(short_link):
    review_urls_list = []
    parsed_url = urlparse(short_link)

    path_parts = parsed_url.path.split('/') 
    # print(path_parts)
    product_name = path_parts[1]
    product_id = path_parts[3]
    for i in range(2,11):
        review_link = f"{parsed_url.scheme}://{parsed_url.netloc}/{product_name}/product-reviews/{product_id}/ref=cm_cr_arp_d_paging_btm_next_{i}?ie=UTF8&reviewerType=all_reviews&pageNumber={i}"
        review_urls_list.append(review_link)
    # review_urls_list = review_urls_list.tolist()
    # print(type(review_urls_list))
    return review_urls_list

In [14]:
available.loc[:, 'reviews2-10'] = available['short_url'].apply(generate_reviews2_10)
available.head()
available['reviews2-10'].iloc[0]

['https://www.amazon.com/EARGO-Hearing-Aids-Adults-Rechargeable/product-reviews/B09NWCSS86/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=2',
 'https://www.amazon.com/EARGO-Hearing-Aids-Adults-Rechargeable/product-reviews/B09NWCSS86/ref=cm_cr_arp_d_paging_btm_next_3?ie=UTF8&reviewerType=all_reviews&pageNumber=3',
 'https://www.amazon.com/EARGO-Hearing-Aids-Adults-Rechargeable/product-reviews/B09NWCSS86/ref=cm_cr_arp_d_paging_btm_next_4?ie=UTF8&reviewerType=all_reviews&pageNumber=4',
 'https://www.amazon.com/EARGO-Hearing-Aids-Adults-Rechargeable/product-reviews/B09NWCSS86/ref=cm_cr_arp_d_paging_btm_next_5?ie=UTF8&reviewerType=all_reviews&pageNumber=5',
 'https://www.amazon.com/EARGO-Hearing-Aids-Adults-Rechargeable/product-reviews/B09NWCSS86/ref=cm_cr_arp_d_paging_btm_next_6?ie=UTF8&reviewerType=all_reviews&pageNumber=6',
 'https://www.amazon.com/EARGO-Hearing-Aids-Adults-Rechargeable/product-reviews/B09NWCSS86/ref=cm_cr_arp_d_paging_btm_next_7?ie=UTF8&re

In [15]:
available.head()


,Unique ID,Product Name,Model,Company,search,amazon_link,short_url,product_name,product_id,review1,reviews2-10
11,8,Eargo,Eargo 6,Eargo 6,EargoEargo 6,https://www.amazon.com/EARGO-Hearing-Aids-Adul...,https://www.amazon.com/EARGO-Hearing-Aids-Adul...,EARGO-Hearing-Aids-Adults-Rechargeable,B09NWCSS86,https://www.amazon.com/EARGO-Hearing-Aids-Adul...,[https://www.amazon.com/EARGO-Hearing-Aids-Adu...
12,9,Eargo,Eargo 7,Eargo 7,EargoEargo 7,https://www.amazon.com/EARGO-Self-Fitting-FDA-...,https://www.amazon.com/EARGO-Self-Fitting-FDA-...,EARGO-Self-Fitting-FDA-Cleared-Hearing-Aids,B0BSHCNP5W,https://www.amazon.com/EARGO-Self-Fitting-FDA-...,[https://www.amazon.com/EARGO-Self-Fitting-FDA...
14,11,Lexie,Lexie B1,hearX Group,LexieLexie B1,https://www.amazon.com/Lexie-Hearing-Aids-Powe...,https://www.amazon.com/Lexie-Hearing-Aids-Powe...,Lexie-Hearing-Aids-Powered-Bluetooth-enabled,B0BHHJWPH7,https://www.amazon.com/Lexie-Hearing-Aids-Powe...,[https://www.amazon.com/Lexie-Hearing-Aids-Pow...
15,12,Lexie,Lexie B2,hearX Group,LexieLexie B2,https://www.amazon.com/Lexie-Rechargeable-Self...,https://www.amazon.com/Lexie-Rechargeable-Self...,Lexie-Rechargeable-Self-fitting-Invisible-Redu...,B0BJ5VBY94,https://www.amazon.com/Lexie-Rechargeable-Self...,[https://www.amazon.com/Lexie-Rechargeable-Sel...
16,13,IQbuds2 MAX,IQbuds2 MAX,Nuheara Limited,IQbuds2 MAXIQbuds2 MAX,https://www.amazon.com/Nuheara-Bluetooth-Recha...,https://www.amazon.com/Nuheara-Bluetooth-Recha...,Nuheara-Bluetooth-Rechargeable-Personalization...,B09J2PVRKC,https://www.amazon.com/Nuheara-Bluetooth-Recha...,[https://www.amazon.com/Nuheara-Bluetooth-Rech...


In [155]:

# output_file_path = 'amazon_urls.csv'
# available.to_csv(output_file_path, index=False) 




## Crawl 

In [ ]:
# links is the same as available df 
# links = pd.read_csv('/Users/alisa/Desktop/PittNAIL/cleardashboard_reviews/amazon_urls.csv')
# links.head()

In [2]:
from selenium import webdriver
import time

In [161]:
# init_links = links['short_url'].copy()
# first_revpg = links['review1'].copy()
# reviews2_10 = links['reviews2-10'].copy()


In [16]:
# def split_urls(urls_string):
#     return eval(urls_string)

# # print(links['reviews2-10'][0])
# print(type(links['reviews2-10'][0]))
# # links['reviews2-10']
# links['reviews2-10'] = links['reviews2-10'].apply(split_urls)
test = available.head(4).copy()

test['reviews2-10'].iloc[0]
# print(type(test['reviews2-10']))
# test.head()

['https://www.amazon.com/EARGO-Hearing-Aids-Adults-Rechargeable/product-reviews/B09NWCSS86/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=2',
 'https://www.amazon.com/EARGO-Hearing-Aids-Adults-Rechargeable/product-reviews/B09NWCSS86/ref=cm_cr_arp_d_paging_btm_next_3?ie=UTF8&reviewerType=all_reviews&pageNumber=3',
 'https://www.amazon.com/EARGO-Hearing-Aids-Adults-Rechargeable/product-reviews/B09NWCSS86/ref=cm_cr_arp_d_paging_btm_next_4?ie=UTF8&reviewerType=all_reviews&pageNumber=4',
 'https://www.amazon.com/EARGO-Hearing-Aids-Adults-Rechargeable/product-reviews/B09NWCSS86/ref=cm_cr_arp_d_paging_btm_next_5?ie=UTF8&reviewerType=all_reviews&pageNumber=5',
 'https://www.amazon.com/EARGO-Hearing-Aids-Adults-Rechargeable/product-reviews/B09NWCSS86/ref=cm_cr_arp_d_paging_btm_next_6?ie=UTF8&reviewerType=all_reviews&pageNumber=6',
 'https://www.amazon.com/EARGO-Hearing-Aids-Adults-Rechargeable/product-reviews/B09NWCSS86/ref=cm_cr_arp_d_paging_btm_next_7?ie=UTF8&re

In [17]:
def crawl(driver, link, filename, directory):
    full_path = os.path.join(directory, f'{filename}.html')
    driver.get('https://www.amazon.com')
    time.sleep(random.randint(6, 22))
    driver.get(link) 
    time.sleep(random.randint(10, 20))
    html_source=driver.page_source
    with open(full_path, 'w', encoding='utf-8') as file:
        file.write(html_source)
    time.sleep(random.randint(10, 16))

    

In [24]:
i=0

driver = webdriver.Chrome()
driver.get('https://www.amazon.com')
time.sleep(20)

for index, row in available.iterrows():
    row_uid = row.loc['Unique ID']
    init_link = row.loc['short_url']
    first_revpg = row.loc['review1']
    reviews2_10 = row.loc['reviews2-10']
    parent_directory = f'/Users/alisa/Desktop/PittNAIL/cleardashboard_reviews/amazon pages'

    directory = os.path.join(parent_directory, str(row_uid))
    # reviews2_10 = ast.literal_eval(reviews2_10) #turning into a list 
    if not os.path.isdir(directory):
        os.makedirs(directory, exist_ok=True)
        crawl(driver, init_link, f'{row_uid}_init', directory)
        print(f"successfully downloaded page: {row_uid}_init")
        full_path = os.path.join(directory, f'{row_uid}_revpg1.html')
        crawl(driver, first_revpg, f'{row_uid}_revpg1', directory)
        print(f"successfully downloaded page: {row_uid}_revpg1")
        for index, revpage in enumerate(reviews2_10):
            index +=2
            full_path = os.path.join(directory, f'{row_uid}_revpg{index}.html')
            crawl(driver, revpage, f'{row_uid}_revpg{index}', directory)
            print(f"successfully downloaded page: {row_uid}_revpg{index}")
            # get all the other review pages
          
     
    i+=1
     

successfully downloaded page: 148_init
successfully downloaded page: 148_revpg1
successfully downloaded page: 148_revpg2
successfully downloaded page: 148_revpg3
successfully downloaded page: 148_revpg4
successfully downloaded page: 148_revpg5
successfully downloaded page: 148_revpg6
successfully downloaded page: 148_revpg7
successfully downloaded page: 148_revpg8
successfully downloaded page: 148_revpg9
successfully downloaded page: 148_revpg10
